# Analyse Objects

 work with Weakly_2024_07
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/12/29
- update : 2024/03/07  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
# december 2023 w_2023_44
fileobjects_in = "objectTable_202312.csv"
fileobjects_out = "objectTableselected_202312.csv"
# Feb 2024 w_2024_07
fileobjects_in = "objectTable_202402.csv"
fileobjects_out = "objectTableselected_202402.csv"

In [ ]:
#tract_selected = 5615
#patches_selected = [294, 313]


tract_selected = 3864
patches_selected = [236, 237]

In [ ]:
suptitle = f"Objects tract = {tract_selected}"

In [ ]:
df = pd.read_csv(fileobjects_in)

In [ ]:
df

In [ ]:
#colnames = sorted(list(df.columns))
#for name in colnames:
#    print(name)

In [ ]:
df_sel = df[['objectId','tabid_idx','x','y','coord_ra','coord_dec','tract','patch','refExtendedness','detect_isPrimary','detect_isIsolated',
            'g_calibFlux','g_psfFlux','g_cModelFlux','g_kronFlux',
            'r_calibFlux','r_psfFlux','r_cModelFlux','r_kronFlux',
            'i_calibFlux','i_psfFlux','i_cModelFlux','i_kronFlux']]

In [ ]:
df_sel.dropna(axis=0,inplace = True)

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
df_sel["g_psfMag"] =  df_sel['g_psfFlux'].map(convert_fluxtomag)
df_sel["r_psfMag"] =  df_sel['r_psfFlux'].map(convert_fluxtomag)
df_sel["i_psfMag"] =  df_sel['i_psfFlux'].map(convert_fluxtomag)
#df_sel["gaussianMag"] =  df_sel['gaussianFlux'].map(lambda x:-2.50 * np.log10(x) + 31.4)
df_sel["g_calibMag"] =  df_sel['g_calibFlux'].map(convert_fluxtomag)
df_sel["r_calibMag"] =  df_sel['r_calibFlux'].map(convert_fluxtomag)
df_sel["i_calibMag"] =  df_sel['i_calibFlux'].map(convert_fluxtomag)

df_sel["g_cModelMag"] =  df_sel['g_cModelFlux'].map(convert_fluxtomag)
df_sel["r_cModelMag"] =  df_sel['r_cModelFlux'].map(convert_fluxtomag)
df_sel["i_cModelMag"] =  df_sel['i_cModelFlux'].map(convert_fluxtomag)

df_sel["g_kronMag"] =  df_sel['g_kronFlux'].map(convert_fluxtomag)
df_sel["r_kronMag"] =  df_sel['r_kronFlux'].map(convert_fluxtomag)
df_sel["i_kronMag"] =  df_sel['i_kronFlux'].map(convert_fluxtomag)

In [ ]:
selection1 = df_sel['refExtendedness']==False
selection2 = df_sel['detect_isPrimary']
selection3 = df_sel['tract'] == tract_selected
selection4 = df_sel['r_cModelMag'] < 16
selection5 = df_sel['patch'] == patches_selected[0]
selection6 = df_sel['patch'] == patches_selected[1]

df_sel = df_sel[selection1 & selection2 & selection3 & selection4 ]
#df_sel = df_sel[selection1 & selection2 & selection3 & selection4 & (selection5 | selection6) ]

In [ ]:
df_sel.dropna(axis=0,inplace = True)

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))
df_sel["g_psfMag"].plot(kind="hist",bins=50,range=(5,30),facecolor="g",logy=True,grid=True,title="psfMag",ax=ax1)
df_sel['g_calibMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="g",logy=True,grid=True,title="calibMag",ax=ax2)
df_sel['g_cModelMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="g",logy=True,grid=True,title="cModelMag",ax=ax3)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))
df_sel["r_psfMag"].plot(kind="hist",bins=50,range=(5,30),facecolor="r",logy=True,grid=True,title="psfMag",ax=ax1)
df_sel['r_calibMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="r",logy=True,grid=True,title="calibMag",ax=ax2)
df_sel['r_cModelMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="r",logy=True,grid=True,title="cModelMag",ax=ax3)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))
df_sel["i_psfMag"].plot(kind="hist",bins=50,range=(5,30),facecolor="y",logy=True,grid=True,title="psfMag",ax=ax1)
df_sel['i_calibMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="y",logy=True,grid=True,title="calibMag",ax=ax2)
df_sel['i_cModelMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="y",logy=True,grid=True,title="cModelMag",ax=ax3)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,10))
df_sel.plot.scatter('coord_ra','coord_dec',c='patch',colormap='hsv',grid=True,ax=ax)
plt.gca().set_aspect('equal')
plt.suptitle(suptitle,fontsize=16)

In [ ]:
df_sel['coord_ra'].plot(kind="hist",bins=50 ,range=(0,360),logy=True,title="coord_ra",grid=True,figsize=(10,5))

In [ ]:
df_sel['coord_dec'].plot(kind="hist",bins=50 ,range=(-90,10),logy=True,title="coord_dec",grid=True,figsize=(10,5))

In [ ]:
df_sel["patch"].unique()

In [ ]:
df_sel = df_sel[ (selection5 | selection6) ]

In [ ]:
df_sel.sort_values(by=["r_cModelMag"] ,ascending=True, inplace=True)

In [ ]:
df_sel

In [ ]:
df_sel.to_csv(fileobjects_out)